In [1]:
import pandas as pd

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/tlp/conditional-tvaes.csv')

In [3]:
df.head()

,demo-genero,demo-fechaEvaluacion,demo-fechaNacimiento,demo-rangoEdad,demo-edad,clin-reservaCognitiva_total,clin-reservaCognitiva_idiomas,clin-reservaCognitiva_ocupacion,clin-reservaCognitiva_escolaridad,clin-ansiedad,...,eval-TLP-Tavec-25_Discriminabilidad-PD,eval-TLP-Tavec-26_Sesgo-PD,eval-TLP-Tavec-27_RI_B_frente_RI_A1-PD,eval-TLP-Tavec-28_RL_CP_frente_RI_A5-PD,eval-TLP-Tavec-29_RCl_Cp_frente_RCl_LP-PD,eval-TLP-Tavec-30_RL_LP_frente_RL_CP-PD,eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD,eval-TLP-Tavec-32_Recon_Ac_frente_RL_LP-PD,eval-TLP-Tavec-33_Recon_Ac_frente_RCl_LP-PD,ED_2Clases
0,2,2024-04-01T02:57:09,1957-08-23T13:14:06,Rango2,69,16,0,2,4,True,...,98.62,0.03,451.46,1983.23,-50.00,662.15,5.19,-0.90,-11.62,D
1,2,2024-03-03T18:56:48,1948-03-17T08:37:02,Rango2,76,18,1,1,5,True,...,89.10,0.48,61.55,5865.54,-50.00,3023.67,-10.46,-29.40,-19.61,D
2,1,2024-03-13T14:27:03,1950-06-24T21:56:13,Rango3,66,21,1,3,5,True,...,94.22,0.31,141.23,-100.00,-15.02,83.85,-8.94,-9.28,-19.99,D
3,2,2024-05-27T21:25:12,1956-11-05T18:38:57,Rango2,68,13,0,0,3,True,...,90.04,0.54,-71.43,2637.20,-50.00,-100.00,-0.80,-60.41,-20.56,D
4,1,2024-04-30T03:24:45,1950-08-11T12:09:27,Rango2,64,16,1,2,4,False,...,86.35,-0.46,456.97,734.99,-50.00,-100.00,6.49,-20.69,-21.72,D


In [4]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((4800, 77), (1200, 77))

In [5]:
predictor= TabularPredictor(
    label='ED_2Clases',
    problem_type='binary',
    eval_metric='roc_auc',
    sample_weight='balance_weight',
    path='AutogluonModels/synthetic/tvaes/v2'
    )

In [6]:
predictor.fit(
    train_data = df_train,
    presets = ['high_quality'],
    time_limit = 1 * 3600,
    auto_stack = True,
    excluded_model_types=['KNN','RF','XT', 'LR'],
    verbosity = 2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       11.73 GB / 15.94 GB (73.6%)
Disk Space Avail:   92.38 GB / 446.36 GB (20.7%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and

(_ray_fit pid=24448) [1000]	valid_set's binary_logloss: 0.389774


(_dystack pid=13492) 	0.8982	 = Validation score   (roc_auc)
(_dystack pid=13492) 	7.24s	 = Training   runtime
(_dystack pid=13492) 	0.38s	 = Validation runtime
(_dystack pid=13492) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 450.95s of the 749.44s of remaining time.
(_dystack pid=13492) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.33%)
(_ray_fit pid=24192) No improvement since epoch 1: early stopping
(_dystack pid=13492) 	0.865	 = Validation score   (roc_auc)
(_dystack pid=13492) 	17.42s	 = Training   runtime
(_dystack pid=13492) 	0.26s	 = Validation runtime
(_dystack pid=13492) Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 430.12s of the 728.61s of remaining time.
(_dystack pid=13492) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=2.14%)
(_dystack pid=13492) 	0.8961	 = Validation 

(_ray_fit pid=23944) [1000]	valid_set's binary_logloss: 0.416472 [repeated 7x across cluster]
(_ray_fit pid=19652) [4000]	valid_set's binary_logloss: 0.37888 [repeated 24x across cluster]


(_dystack pid=13492) 	0.9054	 = Validation score   (roc_auc)
(_dystack pid=13492) 	8.89s	 = Training   runtime
(_dystack pid=13492) 	1.11s	 = Validation runtime
(_dystack pid=13492) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 272.00s of the 570.49s of remaining time.
(_dystack pid=13492) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.17%)
(_dystack pid=13492) 	Warning: Exception caused NeuralNetTorch_r22_BAG_L1 to fail during training... Skipping this model.
(_dystack pid=13492) 		ray::_ray_fit() (pid=19516, ip=127.0.0.1)
(_dystack pid=13492)   File "python\ray\_raylet.pyx", line 1889, in ray._raylet.execute_task
(_dystack pid=13492)   File "c:\Users\jgala\uned\tfm\synthetic-data\.venv\lib\site-packages\autogluon\core\models\ensemble\fold_fitting_strategy.py", line 413, in _ray_fit
(_dystack pid=13492)     fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time

In [7]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                 WeightedEnsemble_L2   0.911074     roc_auc       2.897264  146.529078                0.000000           0.129485            2      False         52
1                 WeightedEnsemble_L3   0.911033     roc_auc       3.299675  387.538390                0.000000           0.166321            3      False         88
2                CatBoost_r137_BAG_L2   0.910302     roc_auc       3.064874  326.876730                0.067865          23.250679            2      False         65
3                 CatBoost_r70_BAG_L2   0.910039     roc_auc       3.087677  354.028247                0.090668          50.402197            2      False         79
4                 CatBoost_r13_BAG_L2   0.909949     roc_auc       3.087793  349.611053                0.090

{'model_types': {'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r9_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r96_BAG_L1': 'StackerEnsembleModel_LGB',
  'XGBoost_r33_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'CatBoost_r137_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_r102_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r13_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r188_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r145_BAG_L1': 'StackerEnsembleModel_NNFast

In [8]:
auxiliary_metrics = ['accuracy', 'balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'roc_auc', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [9]:
predictor.evaluate(
    df_test,
    silent = False,
    auxiliary_metrics = auxiliary_metrics
    )

Evaluation: roc_auc on test data: 0.9305451725949125
Evaluations on test data:
{
    "roc_auc": 0.9305451725949125,
    "accuracy": 0.8583333333333333,
    "balanced_accuracy": 0.8589171027510136,
    "mcc": 0.7181935608411317,
    "f1": 0.859504132231405,
    "precision": 0.8346709470304976,
    "recall": 0.8858603066439523
}


{'roc_auc': 0.9305451725949125,
 'accuracy': 0.8583333333333333,
 'balanced_accuracy': 0.8589171027510136,
 'mcc': 0.7181935608411317,
 'f1': 0.859504132231405,
 'precision': 0.8346709470304976,
 'recall': 0.8858603066439523}

In [10]:
predictor.model_best

'WeightedEnsemble_L2_FULL'

In [11]:
predictors = predictor.leaderboard(
    df_test,
    extra_metrics = auxiliary_metrics,
    extra_info=True,
    silent=True
)

In [12]:
predictors[['roc_auc', 'accuracy', 'balanced_accuracy', 'f1', 'precision', 'recall', 'num_features']]

,roc_auc,accuracy,balanced_accuracy,f1,precision,recall,num_features
0,0.931482,0.860833,0.861328,0.861411,0.839806,0.884157,90
1,0.931468,0.858333,0.858917,0.859504,0.834671,0.885860,90
2,0.930545,0.858333,0.858917,0.859504,0.834671,0.885860,7
3,0.930526,0.855000,0.855365,0.854758,0.837971,0.872232,90
4,0.930503,0.857500,0.858101,0.858794,0.833333,0.885860,3
...,...,...,...,...,...,...,...
171,NaN,NaN,NaN,NaN,NaN,NaN,82
172,NaN,NaN,NaN,NaN,NaN,NaN,82
173,NaN,NaN,NaN,NaN,NaN,NaN,82
174,NaN,NaN,NaN,NaN,NaN,NaN,82


In [13]:
predictor.feature_importance(data=df_train)

These features in provided data are not utilized by the predictor and will be ignored: ['clin-Covid_sintomaSarpullido']
Computing feature importance via permutation shuffling for 75 features using 4800 rows with 5 shuffle sets...
	305.5s	= Expected runtime (61.1s per shuffle set)
	222.35s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
eval-TLP-Stroop-palabrasColor-PD,1.066962e-02,4.488318e-04,3.748834e-07,5,1.159377e-02,9.745468e-03
eval-TLP-Stroop-palabras-PD,8.031937e-03,4.375546e-04,1.052719e-06,5,8.932868e-03,7.131006e-03
demo-fechaNacimiento,6.812891e-03,4.072885e-04,1.525448e-06,5,7.651504e-03,5.974278e-03
eval-TLP-CubCorsi-totalDirectos-PD,4.655345e-03,4.493189e-04,1.028531e-05,5,5.580499e-03,3.730191e-03
eval-TLP-Tavec-7_Rg_Rc-PD,4.270716e-03,5.531067e-04,3.301897e-05,5,5.409570e-03,3.131861e-03
...,...,...,...,...,...,...
ques-MMSE-concentracion-PD,8.906511e-05,7.010527e-06,4.568506e-06,5,1.034999e-04,7.463034e-05
ques-MMSE-orientacion-PD,8.288438e-05,3.936201e-05,4.625060e-03,5,1.639313e-04,1.837446e-06
clin-cardiopatiaIsquemica,7.944678e-05,1.813047e-05,3.040476e-04,5,1.167777e-04,4.211589e-05
ques-MMSE-lenguaje-PD,1.677133e-05,1.242515e-05,1.961592e-02,5,4.235488e-05,-8.812226e-06
